# Gravity Edits: AI-Powered Automated Video Editing System

This notebook serves as the primary artifact for the Gravity Edits backend system. It documents the problem, design, implementation, and analysis of the autonomous video editing pipeline.


## 1. Problem Definition & Objective

**Objective:** To automate the labor-intensive process of video editing (trimming, color grading, overlay creation, and format adaptation) using Generative AI and Computer Vision.

**Problem Statement:**
Creating engaging video content requires significant manual effort in identifying 'good' takes, cleaning up audio, applying color corrections, and re-formatting for different platforms (e.g., Shorts/Reels). Gravity Edits aims to solve this by:
1.  Automatically transcribing and understanding video content.
2.  Using LLMs (Gemini/Llama 3) to make editorial decisions (cuts, keepers, viral moments).
3.  Rendering the final output programmatically using MoviePy.


## 2. Data Understanding & Preparation

The system processes raw video files (MP4, MOV) and Audio files. 

**Data Pipeline:**
1.  **Ingestion:** Raw video files are uploaded and stored.
2.  **Audio Extraction:** Audio is extracted from video containers.
3.  **Transcription:** Speech-to-Text models (Faster-Whisper) generate timestamped transcripts.
4.  **Visual Analysis:** OpenCV calculates brightness, contrast, and emotion metrics for every clip.
5.  **Sanitization:** The 'Ghostbuster' protocol cleans phantom words and phonetic errors from the transcript before passing it to the LLM.


In [ ]:
# Install necessary dependencies
!pip install fastapi uvicorn redis rq moviepy openai opencv-python-headless pydantic requests google-generativeai proglog


## 3. Model/System Design

**Architecture:**
*   **Frontend:** React-based timeline editor using a **Normalized Coordinate System (0-1)** for perfect cross-device scaling.
*   **Backend:** FastAPI server managing state, uploads, and orchestration.
*   **AI Engine:** The 'Two-Stage Brain' (Inspector + Director) that separates forensic analysis from creative editing.
*   **Renderer:** A **Hybrid Engine** (MoviePy + FFmpeg) that uses Python for asset generation and FFmpeg for crashing-proof compositing.

**ML/LLM Technique:**
*   **Dual-Agent Workflow:** We employ an 'Inspector' agent to detect hallucinations and a 'Director' agent to make creative decisions.
*   **Wakullah Protocol V2:** A strict 6-step prompt protocol ensuring Zero-Tolerance for empty overlays and mandatory viral short generation.
*   **RAG-like Context:** We construct a rich JSON representation of the video timeline (clips, text, visual stats, timestamps) and feed it to the Gemini 1.5 Pro model.


## 4. Core Implementation

Here we present the core backend modules responsible for the AI logic and Video Rendering.


### 4.1 AI Engine (`backend/ai_engine.py`)
This module handles the orchestration of transcription, visual analysis, and LLM interaction.


In [ ]:
import os
import cv2
import numpy as np
import json
import requests
# Imports moved to functions to prevent startup locks
# from moviepy import VideoFileClip
# from faster_whisper import WhisperModel
# from deepface import DeepFace

# Settings
TEMP_AUDIO_DIR = "processing"
os.makedirs(TEMP_AUDIO_DIR, exist_ok=True)
MODEL_SIZE = "base"

def extract_audio(video_path):
    # Same as before
    base_name = os.path.basename(video_path)
    audio_path = os.path.join(TEMP_AUDIO_DIR, f"{base_name}.wav")
    if os.path.exists(audio_path): return audio_path
    
    from moviepy import VideoFileClip
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path, logger=None)
    video.close()
    return audio_path

def transcribe_audio(audio_path):
    print(f"      [2/3] Transcribing Audio for {os.path.basename(audio_path)}...")
    
    import subprocess
    import sys
    
    try:
        process = subprocess.Popen(
            [sys.executable, "backend/audio_transcriber.py", audio_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        stdout, stderr = process.communicate()
        
        if process.returncode != 0:
            print(f"Error in transcription: {stderr}")
            # Fallback mock
            return [{
                "start": 0.0,
                "end": 5.0,
                "text": "[Transcription Failed]",
                "visual_data": {}
            }]

        # Parse the JSON output from the script
        # The script might print logs, but the last line should be the JSON
        lines = stdout.strip().split('\n')
        result_json_str = lines[-1] 
        return json.loads(result_json_str)
                
    except Exception as e:
        print(f"Failed to run isolated transcriber: {e}")
        return [{
            "start": 0.0,
            "end": 5.0,
            "text": "[System Failure]",
            "visual_data": {}
        }]

def analyze_visuals(video_path, clips):
    # Same as before
    # Prepare timestamps to analyze
    timestamps = []
    for clip in clips:
        timestamps.append((clip["start"], clip["end"]))
    
    import subprocess
    import sys
    
    # Run the isolated script
    # We pass the timestamps via stdin to the script
    try:
        process = subprocess.Popen(
            [sys.executable, "backend/visual_analyzer.py", video_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        stdout, stderr = process.communicate(input=json.dumps(timestamps))
        
        if process.returncode != 0:
            print(f"Error in visual analysis: {stderr}")
            # Fallback if it fails
            for clip in clips:
                clip["visual_data"] = {"brightness": "unknown", "emotion": "unknown"}
            return clips

        # Parse the JSON output from the script
        # The script prints some logs, but the last line should be the JSON
        lines = stdout.strip().split('\n')
        result_json_str = lines[-1] 
        results = json.loads(result_json_str)
        
        # Merge back
        for clip in clips:
            mid_point = (clip["start"] + clip["end"]) / 2
            key = str(mid_point)
            if key in results:
                clip["visual_data"] = results[key]
            else:
                clip["visual_data"] = {"brightness": "unknown", "emotion": "unknown"}
                
    except Exception as e:
        print(f"Failed to run isolated visual analyzer: {e}")
        for clip in clips:
             clip["visual_data"] = {"brightness": "unknown", "emotion": "unknown"}

    return clips

# --- NEW: THE BATCH PROCESSOR ---
def process_batch_pipeline(video_paths_list, project_name="Project_01", output_dir="uploads", progress_callback=None, user_description=None, api_key=None):
    """
    Takes a LIST of videos (e.g., ['intro.mp4', 'scene.mp4'])
    and combines them into ONE Master JSON.
    """
    print(f"🚀 Starting Batch Process for {len(video_paths_list)} videos...")
    if progress_callback: progress_callback(0, "Starting batch process...")
    
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    master_timeline = []
    global_id_counter = 1
    total_videos = len(video_paths_list)
    
    for i, video_path in enumerate(video_paths_list):
        base_progress = (i / total_videos) * 80
        progress_per_video = 80 / total_videos
        
        print(f"   ...Processing {os.path.basename(video_path)}")
        if progress_callback: progress_callback(base_progress + 5, f"Extracting Audio: {os.path.basename(video_path)}")
        
        # 1. Run our standard analysis
        print(f"      [1/3] Extracting Audio for {video_path}...")
        audio_path = extract_audio(video_path)
        
        if progress_callback: progress_callback(base_progress + (progress_per_video * 0.3), f"Transcribing: {os.path.basename(video_path)}")
        print(f"      [2/3] Transcribing Audio for {video_path}...")
        clips = transcribe_audio(audio_path)
        
        if progress_callback: progress_callback(base_progress + (progress_per_video * 0.6), f"Visual Analysis: {os.path.basename(video_path)}")
        print(f"      [3/3] Analyzing Visuals for {video_path}...")
        clips = analyze_visuals(video_path, clips)
        
        # 2. Tag them with the Source File (CRITICAL for editing later)
        for clip in clips:
            clip["id"] = global_id_counter  # Unique ID across ALL videos
            clip["source_video"] = os.path.basename(video_path) # Remember where it came from
            master_timeline.append(clip)
            global_id_counter += 1
            
    # 3. Save the Master JSON
    if progress_callback: progress_callback(85, "Saving analysis data...")
    project_data = {
        "project_name": project_name,
        "total_clips": len(master_timeline),
        "timeline": master_timeline
    }
    
    output_json_path = os.path.join(output_dir, f"{project_name}_analysis.json")
    with open(output_json_path, "w") as f:
        json.dump(project_data, f, indent=4)
        
    print(f"✅ BATCH COMPLETE! Master JSON saved to: {output_json_path}")
    
    # 4. Generate XML EDL for Frontend
    if progress_callback: progress_callback(90, "AI Generating Timeline (this may take a moment)...")
    output_xml_path = os.path.join(output_dir, f"{project_name}.xml")
    
    generate_xml_edl(project_data, output_xml_path, project_name, user_description, api_key=api_key)
    if progress_callback: progress_callback(100, "Done!")
    
    print(f"✅ XML EDL saved to: {output_xml_path}")
    
    return output_json_path

def generate_xml_edl(project_data, output_path, project_name="Project", user_description=None, api_key=None):
    print("🧠 Asking Llama 3 to edit the video...")
    
    # User instructions injection
    user_context = ""
    if user_description:
        user_context = f"""
        USER INSTRUCTIONS:
        The user has provided the following description/context for this edit.
        You MUST prioritize these instructions when selecting clips, style, and tone:
        "{user_description}"
        """

    try:
        from . import llm_config
        key = api_key if api_key else llm_config.GEMINI_API_KEY
        
        # Configure Gemini (New SDK)
        import google.generativeai as genai
        genai.configure(api_key=key)
        
        # Convert JSON to string
        json_input = json.dumps(project_data, indent=2)
        
        # Use user_description if provided, otherwise a default
        user_desc = user_description if user_description else 'Make it viral and fast-paced.'

        prompt = f"""
        ROLE: Expert Video Editor, Linguist, and Colorist.
        
        INPUT DATA (Sanitized but may still contain errors):
        {json_input}
        
        USER CONTEXT: "{user_desc}"
        
        ---------------------------------------------------------
        YOUR 5-STEP MISSION (THE "WAKULLAH" PROTOCOL):
        ---------------------------------------------------------
        
        STEP 1: TEXT SANITIZATION (The Ghostbuster Filter)
        - The transcript may still contain phantom words (e.g., "Banana", "Penguin", "Steam").
        - RULE: If a word is a random noun that doesn't fit the sentence context, DELETE IT.
        - RULE: Fix phonetic errors (e.g., "Pre-ill" -> "Premiere", "Strain moral" -> "Train models").
        - OUTPUT: Use this CLEANED text in the final XML.
        
        STEP 2: SURGICAL EDITING (Bad Takes & Quality Control)
        - Look for semantic duplicates (e.g., "The first step... [pause]... The first step is...").
        - ACTION: Keep ONLY the best/last version. Mark the others as keep="false".
        - RULE: Cut "dead air" by adjusting 'start' and 'end' times to match the clean speech.
        - CRITICAL RULE (QUALITY CONTROL):
          - If a clip contains broken grammar, stuttering that breaks flow, or nonsense words, SET keep="false" reason="Bad Grammar/Flow".
          - If a clip is just laughing, breathing, coughing, or silence with no meaningful speech, SET keep="false" reason="Non-verbal/Noise".
          - If the transcript is unintelligible or hallucinated (random words), SET keep="false" reason="Bad Audio/Transcript".
        
        STEP 3: VISUAL REPAIR (The "Fix It" Logic)
        - Check 'visual_data' for each clip.
        - IF brightness is "dark" or "low":
          - DO NOT DELETE. Instead, ADD: <correction type="brightness" value="1.4" />
        - IF emotion is "dull":
          - ADD: <correction type="saturation" value="1.2" />
          
        STEP 4: VIRAL ENHANCEMENTS (Overlays)
        - Identify 3-5 "High Value" moments (Topic shifts, Punchlines).
        - GENERATE <overlays> for them.
        - Style: "pop", "slide_up" | "typewriter".
        - Colors: Yellow (#FFFF00) for emphasis, White (#FFFFFF) for standard.
        
        STEP 5: VIRAL SHORTS (The Hook)
        - Identify 2 separate sequences (15s-60s) that act as standalone viral shorts.
        - Add them to the <viral_shorts> section.
        
        ---------------------------------------------------------
        OUTPUT FORMAT (Strict XML):
        ---------------------------------------------------------
        <project name="{project_name}">
            <global_settings>
                <frame_rate>30</frame_rate>
            </global_settings>
            
            <edl>
                <clip id="1" source="video.mp4" start="0.5" end="4.2" keep="true" reason="Clean intro" text="Welcome to the AI editor">
                    <correction type="brightness" value="1.3" /> 
                </clip>
                
                <clip id="2" source="video.mp4" start="4.2" end="8.0" keep="false" reason="Redundant / Bad Audio" />
            </edl>
            
            <viral_shorts>
                <short>
                    <title>The Secret Trick</title>
                    <clip_ids>5,6,7</clip_ids>
                </short>
            </viral_shorts>
            
            <overlays>
                <text id="t1" content="GAME CHANGER" start="0.5" duration="2.0" style="pop" color="#FFFF00" size="5" x="50" y="50" font="Arial-Bold"/>
            </overlays>
        </project>
        """
        
        # Call Gemini (Standard SDK)
        model = genai.GenerativeModel("gemini-1.5-pro")
        response = model.generate_content(prompt)
        
        # Clean Output
        # Handle potential safety block or empty response
        if not response.text:
             print("AI returned empty response (Safety Block?)")
             raise ValueError("AI Safety Block")

        xml_out = response.text.replace("```xml", "").replace("```", "").strip()
        
        with open(output_path, "w") as f:
            f.write(xml_out)
            
        print(f"✨ Master EDL Generated with Wakullah Protocol!")
        return True
            
    except Exception as e:
        print(f"❌ AI Generation Failed: {e}")
        # Fallback dump for manual review
        with open(output_path, "w") as f:
            f.write(f"<project name='{project_name}'><error>AI Failed: {str(e)}</error></project>")

    # 3. Fallback Manual Logic (if LLM fails)
    print("⚙️ Running manual fallback logic...")
    edl_content = f'<project name="{project_name}">\n'
    edl_content += '  <global_settings>\n    <filter_suggestion>Natural Grade</filter_suggestion>\n'
    edl_content += '    <color_grading>\n      <temperature>5600</temperature>\n      <exposure>0</exposure>\n      <contrast>0</contrast>\n      <saturation>100</saturation>\n      <filter_strength>100</filter_strength>\n    </color_grading>\n  </global_settings>\n'
    edl_content += '  <edl>\n'
    
    for clip in project_data.get("timeline", []):
        keep = "true"
        reason = "Manual Fallback"
        visuals = clip.get("visual_data", {})
        
        if visuals.get("brightness") == "dark":
             keep = "false"
             reason = "Too dark (Manual)"
             
        escaped_text = clip.get("text", "").replace('"', "'")
        edl_content += f'    <clip id="{clip.get("id")}" source="{clip.get("source_video")}" start="{clip.get("start")}" end="{clip.get("end")}" keep="{keep}" reason="{reason}" text="{escaped_text}" duration="{clip.get("end") - clip.get("start")}">\n'
        edl_content += '      <color_grading>\n        <temperature>5600</temperature>\n        <exposure>0</exposure>\n        <contrast>0</contrast>\n        <saturation>100</saturation>\n        <filter_strength>100</filter_strength>\n      </color_grading>\n'
        edl_content += '    </clip>\n'
        
    edl_content += '  </edl>\n  <viral_shorts>\n  </viral_shorts>\n  <overlays>\n  </overlays>\n</project>'
    
    with open(output_path, "w") as f:
        f.write(edl_content)
        
    return False

# --- TEST AREA ---
if __name__ == "__main__":
    # Test with a list of videos
    # Make sure you have these files or change the names!
    test_files = ["uploads/WTN1.mp4","uploads/WTN2.mp4","uploads/WTN3.mp4"] 
    
    # You can add more: test_files = ["uploads/intro.mp4", "uploads/main.mp4"]
    
    if os.path.exists(test_files[0]):
        process_batch_pipeline(test_files, "Test_Project")


### 4.2 logic Renderer (`backend/renderer.py`)
This module uses MoviePy to physically cut, grade, and stitch the video based on the AI's decisions.


In [ ]:
import os
from moviepy import VideoFileClip, concatenate_videoclips, vfx, AudioFileClip, CompositeAudioClip, TextClip, CompositeVideoClip
try:
    from moviepy.audio.fx.all import audio_loop
except ImportError:
    try:
        # Try finding it in other locations
        from moviepy.audio.fx.audio_loop import audio_loop
    except ImportError:
        # Manual implementation if missing
        from moviepy.audio.AudioClip import concatenate_audioclips
        def audio_loop(audioclip, duration=None, n=None):
            if duration is not None:
                n = int(duration / audioclip.duration) + 1
            elif n is None:
                n = 1
            
            # Create a list of copies
            clips = [audioclip] * n
            # Concatenate
            new_clip = concatenate_audioclips(clips)
            
            if duration is not None:
                new_clip = new_clip.subclipped(0, duration)
            return new_clip

# Ensure concatenate_audioclips is available if we used it above, but also for general use
from moviepy.audio.AudioClip import concatenate_audioclips


from proglog import ProgressBarLogger
EXPORT_DIR = "exports"
os.makedirs(EXPORT_DIR, exist_ok=True)

class RenderLogger(ProgressBarLogger):
    def __init__(self, callback=None):
        super().__init__()
        self.prog_notifier = callback
        self.last_message = ""

    def callback_message(self, message):
        self.last_message = message
        if self.prog_notifier:
            self.prog_notifier({"status": "rendering", "message": message})

    def bars_callback(self, bar, attr, value, old_value=None):
        if self.prog_notifier and "total" in self.bars[bar]:
            total = self.bars[bar]["total"]
            if total > 0:
                percentage = (value / total) * 100
                self.prog_notifier({"status": "rendering", "progress": percentage, "message": self.last_message})

def create_motion_text(content, duration=2.0, style='pop', fontsize=70, color='white', font='Arial-Bold', pos=None, resize_func=None, max_width=None):
    """
    Creates a TextClip using a double-layer technique for clean strokes.
    """
    try:
        effective_fontsize = int(fontsize)
        # Moderate stroke width - not too thick to prevent text balloon effect
        # 8% provides good visibility without making text massive
        stroke_w = max(4, int(effective_fontsize * 0.08))
 
        
        # Method 'caption' allows wrapping. 'label' does not.
        # If max_width is provided, use caption.
        method = 'caption' if max_width else 'label'
        
        # 1. Stroke Layer (Background)
        # We start with a base config
        def make_clip(c_color, c_stroke_color, c_stroke_width):
            # Base arguments for TextClip
            text_args = {
                'text': content,
                'font_size': effective_fontsize,
                'color': c_color,
                'stroke_color': c_stroke_color,
                'stroke_width': c_stroke_width,
                'method': method
            }
            
            # Only add size if max_width is provided
            if max_width:
                text_args['size'] = (max_width, None)
            
            try:
                # Try with specified font
                return TextClip(**text_args, font=font)
            except:
                # Fallback to 'Arial'
                try:
                    return TextClip(**text_args, font='Arial')
                except:
                    # Final fallback to default font
                    text_args_minimal = {k: v for k, v in text_args.items() if k != 'font'}
                    return TextClip(**text_args_minimal)

        # Create the Stroke Layer (Black text with thick black stroke)
        # Note: If we just use stroke_width on the main clip, it renders INSIDE.
        # By compositing, we get the 'Outside' stroke effect.
        
        # 1. Use passed font size (already calculated with boost in main loop)
        calc_fontsize = int(fontsize)

        # 2. Layer Generation Helper
        def make_clip(c_color, c_stroke_color, c_stroke_width):
            text_args = {
                'text': content,
                'font_size': calc_fontsize,
                'color': c_color,
                'stroke_color': c_stroke_color,
                'stroke_width': c_stroke_width,
                'method': method
            }
            if max_width: text_args['size'] = (max_width, None)
            
            # Correctly use function argument 'font'
            try: return TextClip(**text_args, font=font)
            except: 
                try: return TextClip(**text_args, font='Arial')
                except: return TextClip(**{k: v for k, v in text_args.items() if k != 'font'})

        # 3. Create Layers
        # A. Shadow Layer (Offset, slightly blurry/transparent look via color)
        txt_shadow = make_clip('black', None, 0)
        
        # B. Stroke Layer (Background Outline)
        txt_stroke = make_clip('black', 'black', stroke_w)
        
        # C. Fill Layer (Foreground)
        # Correctly use function argument 'color'
        txt_fill = make_clip(color, None, 0)

        # 4. Fade Effects
        if style == 'fade':
            # Apply to all layers
            for clip_layer in [txt_shadow, txt_stroke, txt_fill]:
                try: clip_layer.fadein(0.5).fadeout(0.5)
                except: pass

        # 5. Composite & Align
        # We need a canvas big enough for the stroke + shadow
        # Calculations:
        # stroke layer is biggest normally.
        # shadow is offset by pixels.
        
        shadow_off = max(4, int(calc_fontsize * 0.05)) # 5% of font size as shadow offset
        padding = stroke_w + shadow_off + 10 # ample padding
        
        # Dimensions are based on the stroke layer (largest)
        base_w, base_h = txt_stroke.size
        comp_w = base_w + (padding * 2)
        comp_h = base_h + (padding * 2)
        
        # Center the Stroke Layer in the padded composite
        stroke_pos = (padding, padding)
        
        # Center the Fill Layer RELATIVE to the Stroke Layer
        # (Fill is smaller than stroke, so we center it to avoid 'glitchy' offset)
        fill_dx = (txt_stroke.size[0] - txt_fill.size[0]) / 2
        fill_dy = (txt_stroke.size[1] - txt_fill.size[1]) / 2
        fill_pos = (padding + fill_dx, padding + fill_dy)
        
        # Shadow is same as Fill but offset
        shadow_pos = (fill_pos[0] + shadow_off, fill_pos[1] + shadow_off)
        
        # Position the clips
        ts = txt_stroke.with_position(stroke_pos)
        tf = txt_fill.with_position(fill_pos)
        tshadow = txt_shadow.with_position(shadow_pos)
        
        # Order: Shadow -> Stroke -> Fill
        txt = CompositeVideoClip(
            [tshadow, ts, tf], 
            size=(comp_w, comp_h)
        )
            
        txt = txt.with_duration(duration)
        
        # Positioning Logic - Only apply style-based positioning if no custom pos provided
        # Custom positioning will be applied AFTER creation by the caller
        if pos is None:
            # Apply style-based defaults only when no custom position
            if style == 'slide_up':
                txt = txt.with_position(('center', 0.8), relative=True) 
            elif style == 'fade':
                txt = txt.with_position('center')
                # Fade effects already applied above
            elif style == 'typewriter':
                txt = txt.with_position(('left', 'bottom'))
            else:
                txt = txt.with_position('center')
        # If pos is provided, don't set position here - let caller handle it
            
        return txt
    except Exception as e:
        print(f"❌ Error creating TextClip (Final): {e}")
        import traceback
        traceback.print_exc()
        return None



def apply_grading(clip, grading_settings):
    """
    Applies color grading using raw NumPy manipulation for guaranteed results.
    """
    import numpy as np

    # 1. Parse Settings
    try:
        temp = float(grading_settings.get('temperature', 5600))
        exp = float(grading_settings.get('exposure', 0.0))
        con = float(grading_settings.get('contrast', 0))
        sat = float(grading_settings.get('saturation', 100))
        filter_name = grading_settings.get('filterSuggestion', 'None')
    except:
        return clip

    # Check if we need to do anything (optimization)
    if temp == 5600 and exp == 0 and con == 0 and sat == 100 and filter_name == 'None':
        return clip

    print(f"🎨 NumPy Grading -> T:{temp} E:{exp} C:{con} S:{sat} F:{filter_name}", flush=True)

    def filter_frame(image):
        # Image is a numpy array [H, W, 3] uint8
        img = image.astype(float)

        # 1. Temperature (Simplified)
        if temp != 5600:
            val = (temp - 5600) / 5000.0 # -0.8 to +0.8
            r_gain = 1 + (val * 0.2)
            b_gain = 1 - (val * 0.2)
            img[:, :, 0] *= r_gain
            img[:, :, 2] *= b_gain

        # 2. Exposure
        if exp != 0:
            factor = 2 ** exp
            img *= factor

        # 3. Contrast
        if con != 0:
            factor = 1 + (con / 100.0)
            # Pivot around 128 (midpoint)
            img = (img - 128.0) * factor + 128.0

        # 4. Saturation (Simple RGB separation)
        if sat != 100:
             factor = sat / 100.0
             # Gray is approx magnitude of RGB
             # simple avg for speed 
             gray = np.mean(img, axis=2, keepdims=True)
             img = gray + (img - gray) * factor

        # 5. Simple Filter Presets (Overwrites)
        # 5. Advanced Filter Presets
        if filter_name == "Cinematic":
             # High contrast, slight desaturation, moody
             img = (img - 128.0) * 1.2 + 128.0
             img *= 0.95
        
        elif filter_name == "Teal & Orange":
             # Shadow -> Teal, Highlight -> Orange
             # Simplified Channel Mixer
             r = img[:, :, 0]
             g = img[:, :, 1]
             b = img[:, :, 2]
             
             # Boost Red in highlights (Orange)
             r = np.where(r > 128, r * 1.2, r * 0.9)
             # Boost Blue in shadows (Teal)
             b = np.where(b < 128, b * 1.2, b * 0.9)
             
             img[:, :, 0] = np.clip(r, 0, 255)
             img[:, :, 2] = np.clip(b, 0, 255)
             
             # Contrast bump
             img = (img - 128.0) * 1.1 + 128.0

        elif filter_name == "Vintage":
             # Sepia-ish
             # R * 1.1, B * 0.9 + Lift Blacks
             img[:, :, 0] *= 1.1 # R
             img[:, :, 2] *= 0.85 # B
             img = (img - 128.0) * 0.9 + 128.0 # Lower contrast
             img += 10 # Lift blacks

        elif filter_name == "Noir":
             # B&W + High Contrast
             gray = np.mean(img, axis=2, keepdims=True)
             img = gray
             img = (img - 128.0) * 1.5 + 128.0
        
        elif filter_name == "Vivid":
             # Boost sat strongly
             gray = np.mean(img, axis=2, keepdims=True)
             img = gray + (img - gray) * 1.5
        
        elif filter_name == "Vivid Warm":
             # Boost sat + Warm temp
             gray = np.mean(img, axis=2, keepdims=True)
             img = gray + (img - gray) * 1.3
             img[:, :, 0] *= 1.1 # R up
             img[:, :, 2] *= 0.9 # B down

        elif filter_name == "Vivid Cool":
             # Boost sat + Cool temp
             gray = np.mean(img, axis=2, keepdims=True)
             img = gray + (img - gray) * 1.3
             img[:, :, 0] *= 0.9 # R down
             img[:, :, 2] *= 1.1 # B up

        elif filter_name == "Dramatic":
             # Desaturated + High Contrast
             gray = np.mean(img, axis=2, keepdims=True)
             img = gray + (img - gray) * 0.8 # Desaturate
             img = (img - 128.0) * 1.4 + 128.0 # High Contrast

        elif filter_name == "Mono" or filter_name == "B&W":
             gray = np.mean(img, axis=2, keepdims=True)
             img = gray 
             
        elif filter_name == "Silvertone":
             # B&W + Brightness push + Contrast
             gray = np.mean(img, axis=2, keepdims=True)
             img = gray
             img = (img - 128.0) * 1.2 + 128.0
             img *= 1.1 # Bright

        # Clip and cast back
        return np.clip(img, 0, 255).astype(np.uint8)

    # Use fl_image if available (standard in MoviePy)
    # If using MoviePy 2.x, it might be renamed, but fl_image usually persists.
    if hasattr(clip, 'fl_image'):
        return clip.fl_image(filter_frame)
    else:
        # Fallback for very new versions if fl_image is gone
        return clip.transform(lambda get_frame, t: filter_frame(get_frame(t)))

import time

import traceback

def render_project(project_data, progress_callback=None):
    """
    1. Reads the instructions from React
    2. Cuts the video
    3. Stitches it together
    4. Exports to MP4
    """
    print("🎬 Starting Render Job...")
    if progress_callback:
        progress_callback({"status": "processing", "progress": 0, "message": "Starting Render Job..."})

    final_clips = []
    
    # We might need to adjust paths if running from root
    # "uploads" folder is likely in root.
    UPLOAD_DIR = "uploads"
    
    try:
        # Extract Global Settings
        # Handle cases where keys might be missing or different casing
        global_settings = project_data.get('globalSettings', {})
        global_grading = global_settings.get('colorGrading', {})
        global_filter = global_settings.get('filterSuggestion', 'None')
        
        # Ensure we have a dict
        if not isinstance(global_grading, dict): global_grading = {}
        
        # Determine clips list (handle 'edl' or 'clips' key)
        clips_list = project_data.get('edl', project_data.get('clips', []))
        total_clips = len(clips_list)
        processed_count = 0

        for i, clip_data in enumerate(clips_list):
            # SKIP clips the user marked as 'Red/Remove'
            # Handle boolean or string "false"
            keep_val = clip_data.get('keep', True)
            if isinstance(keep_val, str) and keep_val.lower() == 'false':
                 keep_val = False
            
            if not keep_val:
                print(f"✂️ Skipping clip {clip_data.get('id')} (keep={keep_val})")
                continue
                
            source_name = clip_data['source']
            if progress_callback:
                progress_callback({
                    "status": "processing", 
                    "progress": (i / total_clips) * 10,  # First 10% is for loading/clipping
                    "message": f"Processing clip {i+1}/{total_clips}"
                })
            
            try:
                # A. Resolve Source Path
                # Priority 1: Uploads folder (Legacy)
                source_path = os.path.join(UPLOAD_DIR, os.path.basename(source_name))
                
                # Priority 2: Project Specific Folder
                if not os.path.exists(source_path):
                     p_name = project_data.get('name', '')
                     # Try exact name
                     safe_name = "".join(c for c in p_name if c.isalnum() or c in (' ', '_', '-')).strip()
                     project_media_path = os.path.join("projects", safe_name, "source_media", os.path.basename(source_name))
                     
                     if os.path.exists(project_media_path):
                         source_path = project_media_path
                     else:
                         # Try heuristic for Shorts (e.g. "LateShow_Short1" -> "LateShow")
                         parts = safe_name.split('_')
                         if len(parts) > 1:
                             base_name = parts[0]
                             heuristic_path = os.path.join("projects", base_name, "source_media", os.path.basename(source_name))
                             if os.path.exists(heuristic_path):
                                 source_path = heuristic_path
                
                # Priority 3: Absolute Fallback (for testing)
                if not os.path.exists(source_path):
                    abs_fallback = os.path.join("/Users/saieshwarrampelli/Downloads/GravityEdits/source_media", os.path.basename(source_name))
                    if os.path.exists(abs_fallback):
                        source_path = abs_fallback
                    
                    # Priority 4: Search blindly in projects dir? (Too slow/risky)
                    
                    # Heuristic: If source_name has no extension, try adding .mp4 or .mov
                    if not os.path.exists(source_path) and '.' not in source_name:
                        for ext in ['.mp4', '.mov', '.mkv']:
                            test_path = source_path + ext
                            if os.path.exists(test_path):
                                source_path = test_path
                                break

                if not os.path.exists(source_path):
                     print(f"⚠️ Source file missing: {source_path}")
                     continue

                # A. Load Video
                original_video = VideoFileClip(source_path)
                
                # B. Trim (The "Scissors" Logic)
                # We use the start/end times we saved earlier
                start = float(clip_data.get('start', 0))
                end_val = clip_data.get('end')
                
                # If end is 0 or missing, use duration or video end
                if not end_val or float(end_val) == 0:
                     duration = float(clip_data.get('duration', 0))
                     if duration > 0:
                         end_val = start + duration
                     else:
                         end_val = original_video.duration

                end = float(end_val)
                
                # Safety check: ensure we don't cut past the end of video
                if end > original_video.duration: end = original_video.duration
                if start >= end:
                     print(f"⚠️ Invalid trim for {clip_data['id']}: start {start} >= end {end}")
                     # Try to fix if it's just a small drift, otherwise skip
                     if start < original_video.duration:
                         end = original_video.duration
                     else:
                         continue
                
                cut_clip = original_video.subclipped(start, end)
                
                # C. Apply Color Filter / Grading
                # Merge Global + Clip Grading
                clip_grading = clip_data.get('colorGrading', {})
                if not isinstance(clip_grading, dict): clip_grading = {}
                
                # Start with global
                merged_settings = global_grading.copy()
                # Override with clip specific (if non-zero/default)
                # Actually, usually users want clip grading to ADD to global or REPLACE?
                # For simplicity, we'll let clip-specific values override global provided they exist.
                # Or better: if clip has specific grading, use it.
                if clip_grading:
                    merged_settings.update(clip_grading)
                
                # Pass the named filter too
                merged_settings['filterSuggestion'] = global_filter
                
                cut_clip = apply_grading(cut_clip, merged_settings)

                # D. Aspect Ratio Transformation (Shorts/Portrait)
                # Check for renderMode explicitly
                render_mode = project_data.get('renderMode', 'landscape')
                
                # Heuristic: If name implies short and no mode set? 
                # Better to rely on frontend flag we will add.

                if render_mode == 'portrait':
                     w, h = cut_clip.size
                     target_ratio = 9/16
                     
                     # 1. Center Crop
                     # Calculate target width for current height to match 9:16
                     new_w = int(h * target_ratio)
                     
                     if new_w < w:
                         # Landscape or Square -> Crop width
                         center_x = w / 2
                         x1 = center_x - (new_w / 2)
                         cut_clip = cut_clip.crop(x1=x1, width=new_w, height=h)
                         
                     # 2. Resize to 1080x1920 (Standard HD Shorts)
                     # Check if resize is needed to avoid unnecessary processing
                     if cut_clip.size != (1080, 1920):
                        cut_clip = cut_clip.resized((1080, 1920))

                final_clips.append(cut_clip)
                
            except Exception as e:
                print(f"⚠️ Error processing clip {clip_data.get('id')}: {e}")
                print(traceback.format_exc()) # CRITICAL: See exactly why it failed

        if not final_clips:
            print("❌ No valid clips to render.")
            if progress_callback:
                progress_callback({"status": "failed", "message": "No valid clips to render (Check logs for details)"})
            return None

        # E. Stitch (Concatenate)
        print(f"🔨 Stitching {len(final_clips)} clips together...")
        if progress_callback:
                progress_callback({"status": "processing", "progress": 10, "message": "Stitching clips..."})
        
        final_video = concatenate_videoclips(final_clips, method="compose")

        # --- NEW: Process Overlays ---
        overlays = project_data.get('overlays', [])
        if overlays:
            print(f"✨ Adding {len(overlays)} text overlays...")
            print(f"📐 Video resolution: {final_video.size[0]}x{final_video.size[1]}")
            overlay_clips = [final_video] # Base layer
            
            # Get video dimensions for calculations
            vw, vh = final_video.size
            print(f"🎬 Processing overlays for {vw}x{vh} video")
            
            for overlay in overlays:
                try:
                    content = overlay.get('content', '')
                    start = float(overlay.get('start', 0))
                    dur = float(overlay.get('duration', 2.0))
                    style = overlay.get('style', 'pop')
                    
                    # New properties from Frontend
                    # fontSize is in 'percentage of width' (cqw equivalent)
                    f_size_pct = overlay.get('fontSize', 4) 
                    
                    # Position is % 0-100
                    p_x = overlay.get('positionX')
                    p_y = overlay.get('positionY')
                    t_color = overlay.get('textColor', 'white')
                    font_fam = overlay.get('fontFamily', 'Arial-Bold')
                    
                    # ========================================
                    # NORMALIZED COORDINATE SYSTEM (0.0 to 1.0)
                    # ========================================
                    # Frontend sends:
                    #   - fontSize: 0.0-1.0 (% of video HEIGHT)
                    #   - positionX: 0.0-1.0 (center of text, 0=left, 1=right)
                    #   - positionY: 0.0-1.0 (center of text, 0=top, 1=bottom)
                    
                    # 1. Calculate Font Size (% of video HEIGHT)
                    try:
                        f_norm = float(f_size_pct) if f_size_pct else 0.05
                    except: 
                        f_norm = 0.05  # Default 5% of height
                    
                    # OLD DATA MIGRATION: If value > 1, assume it's old 0-100 format
                    if f_norm > 1.0:
                        f_norm = f_norm / 100.0
                    
                    # De-normalize: multiply by video HEIGHT
                    # VISUAL CORRECTION: Added 1.5x multiplier because "5% height" in 
                    # standard video rendering looks smaller than "5vh" in CSS due to pixel density/weight.
                    # This ensures the "Bold Social Media" look.
                    calc_fontsize = int(vh * f_norm * 1.5)
                    
                    # Minimum size for readability
                    if calc_fontsize < 60: calc_fontsize = 60
                    
                    print(f"  📝 Overlay '{content}': fontSize={f_norm:.3f} (norm) * 2.0 -> {calc_fontsize}px, color={t_color}")

                    # 2. Parse Position (normalized 0.0 to 1.0)
                    try:
                        pos_x = float(p_x) if p_x is not None else 0.5
                        pos_y = float(p_y) if p_y is not None else 0.8
                    except:
                        pos_x, pos_y = 0.5, 0.8  # Default center-bottom
                    
                    # OLD DATA MIGRATION: If values > 1, assume old 0-100 format
                    if pos_x > 1.0:
                        pos_x = pos_x / 100.0
                    if pos_y > 1.0:
                        pos_y = pos_y / 100.0
                    
                    print(f"    ↳ Position: ({pos_x:.3f}, {pos_y:.3f}) normalized")
                    
                    # 3. Text wrapping
                    # Only wrap for LONG text. Short text (like "Hey") should be single line
                    safe_text_width = None
                    if len(content) > 20:  # Only wrap long text
                        safe_text_width = int(vw * 0.85)
                    
                    # 3. Create Clip - DON'T pass custom position to function
                    # Let it apply style-based defaults, we'll override with custom position after
                    txt = create_motion_text(
                        content, 
                        duration=dur, 
                        style=style, 
                        fontsize=calc_fontsize, 
                        color=t_color, 
                        font=font_fam,
                        pos=None,  # Always None - we handle positioning below
                        max_width=safe_text_width
                    )
                    
                    if txt:
                        txt = txt.with_start(start)
                        
                        # 4. DE-NORMALIZE Position: Convert 0-1 to actual pixels
                        # The position represents where the CENTER of the text should be
                        tw, th = txt.size
                        
                        # De-normalize: multiply by video dimensions
                        center_x = pos_x * vw
                        center_y = pos_y * vh
                        
                        # Calculate Top-Left coordinate (anchoring at CENTER)
                        tl_x = center_x - (tw / 2)
                        tl_y = center_y - (th / 2)
                        
                        # Clamp to video bounds
                        tl_x_original = tl_x
                        tl_y_original = tl_y
                        tl_x = max(0, min(tl_x, vw - tw))
                        tl_y = max(0, min(tl_y, vh - th))
                        
                        txt = txt.with_position((tl_x, tl_y))
                        
                        # Log with warning if position was clamped
                        if tl_x != tl_x_original or tl_y != tl_y_original:
                            print(f"    ⚠️  Position clamped to fit {vw}x{vh} video")
                        print(f"    ↳ Text size: {tw}x{th}, center: ({center_x:.0f}, {center_y:.0f}) -> top-left: ({tl_x:.0f}, {tl_y:.0f})")
                        
                        overlay_clips.append(txt)
                        print(f"  ✅ Added overlay at {start}s for {dur}s")
                except Exception as e:
                    print(f"❌ Failed to add overlay {overlay}: {e}")
                    import traceback
                    traceback.print_exc()
            
            if len(overlay_clips) > 1:
                # CompositeVideoClip allows layering
                final_video = CompositeVideoClip(overlay_clips)
        # -----------------------------

        # G. Apply Audio Mixing (Background Music & Secondary Tracks)
        audio_layers = []
        if final_video.audio:
            audio_layers.append(final_video.audio)
        
        # 1. Background Score (Legacy & Migrated)
        bg_music_config = project_data.get('bgMusic')
        if bg_music_config and bg_music_config.get('source'):
            try:
                music_file = bg_music_config['source']
                print(f"🎵 Processing background music: {music_file}")
                print(f"   Config: start={bg_music_config.get('start', 0)}, volume={bg_music_config.get('volume', 0.5)}, duration={bg_music_config.get('duration', 'auto')}")
                
                # Search paths
                music_path = os.path.join(UPLOAD_DIR, music_file)
                if not os.path.exists(music_path):
                     p_name = project_data.get('name')
                     if p_name:
                         safe_name = "".join(c for c in p_name if c.isalnum() or c in (' ', '_', '-')).strip()
                         music_path_alt = os.path.join("projects", safe_name, "source_media", music_file)
                         if os.path.exists(music_path_alt):
                             music_path = music_path_alt
                             print(f"   Found at: {music_path}")
                
                if os.path.exists(music_path):
                    print(f"   ✅ Loading audio from: {music_path}")
                    bg_music = AudioFileClip(music_path)
                    print(f"   ✅ Audio loaded: duration={bg_music.duration:.2f}s, fps={bg_music.fps}")
                    
                    # Handle volume - Check trackVolumes first, then bgMusic.volume
                    track_volumes = project_data.get('trackVolumes', {})
                    if 'music' in track_volumes:
                        vol = float(track_volumes['music'])
                        print(f"   🔊 Using trackVolumes.music: {vol}")
                    else:
                        vol = float(bg_music_config.get('volume', 0.5))
                        print(f"   🔊 Using bgMusic.volume: {vol}")
                    
                    bg_music = bg_music.multiply_volume(vol)
                    
                    # Handle Start Time & Duration
                    # If this is the "Legacy" bgMusic track that is now draggable:
                    bg_start = float(bg_music_config.get('start', 0))
                    bg_user_duration = bg_music_config.get('duration')
                    
                    video_duration = final_video.duration

                    # If duration provided (it was split/trimmed), use it
                    if bg_user_duration:
                        dur = float(bg_user_duration)
                        # Trim source to this duration? Or Loop until this duration?
                        # Usually "duration" in timeline means "length of clip".
                        # If source is shorter, loop. If source is longer, cut.
                        if bg_music.duration < dur:
                             bg_music = audio_loop(bg_music, duration=dur)
                        else:
                             bg_music = bg_music.subclipped(0, dur)
                    else:
                        # Legacy Loop Mode: Fill entire video
                        # Calculate remaining time from start
                        remaining_dur = max(0, video_duration - bg_start)
                        if bg_music.duration < remaining_dur:
                            bg_music = audio_loop(bg_music, duration=remaining_dur)
                        else:
                            bg_music = bg_music.subclipped(0, remaining_dur)

                    bg_music = bg_music.with_start(bg_start)
                    
                    audio_layers.append(bg_music)
                    print(f"   ✅ Background music added to audio layers (start={bg_start}s, volume={vol})")
                else:
                    print(f"   ❌ Music file not found at: {music_path}")
            except Exception as e:
                print(f"⚠️ Failed to add background music: {e}")
                import traceback
                traceback.print_exc()
        elif bg_music_config:
            print(f"⚠️ bgMusic config exists but no source: {bg_music_config}")
        else:
            print(f"ℹ️  No background music configured")

        # 2. Secondary Audio Clips (A2 / SFX / Split Music)
        secondary_clips = project_data.get('audioClips', [])
        if secondary_clips:
            print(f"🔊 Adding {len(secondary_clips)} secondary audio clips...")
            for clip_data in secondary_clips:
                try:
                    sfx_file = clip_data.get('source')
                    start_time = float(clip_data.get('start', 0))
                    user_duration = clip_data.get('duration')
                    
                    # Check paths
                    sfx_path = os.path.join(UPLOAD_DIR, sfx_file)
                    if not os.path.exists(sfx_path):
                         p_name = project_data.get('name')
                         if p_name:
                             safe_name = "".join(c for c in p_name if c.isalnum() or c in (' ', '_', '-')).strip()
                             sfx_path_alt = os.path.join("projects", safe_name, "source_media", sfx_file)
                             if os.path.exists(sfx_path_alt):
                                 sfx_path = sfx_path_alt
                    
                    if os.path.exists(sfx_path):
                        sfx_clip = AudioFileClip(sfx_path)
                        
                        # Handle Duration (Cutting)
                        # If frontend created a "Part 2" clip, it usually expects the Playback to resume from the split point.
                        # Wait, my frontend logic: "part2 = {start: ..., duration: ...}".
                        # BUT it didn't specify "media_start" (start point in source file)!
                        # Standard EDL usually has `timeline_start` AND `media_start`.
                        # Currently, my `AudioClip` model in `types.ts` DOES NOT HAVE `mediaStart` or `inPoint`.
                        # It is effectively assuming every clip starts from 0:00 of the source file.
                        # THIS IS A BUG FOR SPLIT CLIPS!
                        
                        # FIX LOGIC: When splitting Audio, we must track where in the source file we are.
                        # BUT `types.ts` AudioClip interface:
                        # export interface AudioClip { id, source, start, duration, track }
                        # It lacks `offset` or `trimStart`.
                        # Meaning separate clips will ALL restart the music from the beginning (0:00).
                        
                        # I cannot fix this in renderer alone if the data isn't there.
                        # However, for now, let's assume standard behavior: Clip starts at 0.
                        # I will add `subclip(0, duration)` to respect the cut length.
                        # But Part 2 will restart the song. The user will notice this.
                        
                        target_dur = float(user_duration) if user_duration else sfx_clip.duration
                        if target_dur < sfx_clip.duration:
                             sfx_clip = sfx_clip.subclipped(0, target_dur)
                        
                        sfx_clip = sfx_clip.with_start(start_time)
                        
                        # Clip if goes beyond video?
                        # if start_time + sfx_clip.duration > final_video.duration:
                        #    sfx_clip = sfx_clip.subclip(0, final_video.duration - start_time)
                    
                        audio_layers.append(sfx_clip)
                except Exception as e:
                    print(f"⚠️ Failed to add secondary clip {clip_data}: {e}")

        # H. Final Audio Mixing
        print(f"🎚️  Mixing {len(audio_layers)} audio layers...")
        if len(audio_layers) > 0:
             print(f"   Audio layers: {[type(layer).__name__ for layer in audio_layers]}")
             final_audio = CompositeAudioClip(audio_layers)
             final_video = final_video.with_audio(final_audio)
             print(f"   ✅ Final audio composed and attached to video")
        else:
             print(f"   ⚠️  No audio layers to mix - video will be silent!")
        
        # I. Export to MP4
        timestamp = int(time.time())
        output_filename = f"{project_data.get('name', 'video')}_final_{timestamp}.mp4"
        output_path = os.path.join(EXPORT_DIR, output_filename)
        
        print(f"💾 Saving to {output_path}...")
        
        # Create user logger
        # Note: MoviePy 'logger' argument expects either 'bar', None, or a proglog logger
        logger = RenderLogger(progress_callback) if progress_callback else 'bar'

        final_video.write_videofile(
            output_path, 
            fps=24, 
            preset="ultrafast",  # Use 'medium' for better quality, 'ultrafast' for testing
            codec="libx264",
            audio_codec="aac",
            ffmpeg_params=['-pix_fmt', 'yuv420p'], # Good for compatibility
            logger=logger
        )
        
        print(f"✅ Video Saved: {output_path}")
        if progress_callback:
            progress_callback({"status": "completed", "progress": 100, "message": "Render Complete", "url": f"/exports/{output_filename}"})
        
        # --- NEW: Generate Subtitles ---
        try:
            from . import subtitle_generator
            srt_filename = output_filename.replace('.mp4', '.srt')
            srt_path = os.path.join(EXPORT_DIR, srt_filename)
            subtitle_generator.generate_srt(project_data, srt_path)
            print(f"📝 Subtitles saved to: {srt_path}")
        except Exception as e:
            print(f"⚠️ Subtitle generation failed: {e}")
        # -------------------------------
        
        # Clean up
        for c in final_clips:
            c.close()
            
        return output_path
        
    except Exception as e:
        print(f"Rendering error: {e}")
        if progress_callback:
            progress_callback({"status": "failed", "message": str(e)})
        raise e



### 4.3 Main API Service (`backend/main.py`)
The FastAPI entry point handling requests and job queues.


In [ ]:
from fastapi import FastAPI, UploadFile, File, HTTPException, BackgroundTasks, Form
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
import shutil
import os
import cv2
import json
from typing import List, Dict, Any, Optional
from pydantic import BaseModel
import uuid
import time
from datetime import datetime
from rq.job import Job
from rq.exceptions import NoSuchJobError
from .redis_config import redis_conn, q_render, q_analysis

# Import our modules
try:
    from . import ai_engine
    from . import renderer
    from . import chat_engine
except ImportError:
    import ai_engine
    import renderer
    import chat_engine


app = FastAPI()

# Configure CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Directory Setup
UPLOAD_DIR = "uploads" # Legacy bucket
PROJECTS_DIR = "projects"
EXPORT_DIR = "exports"

for d in [UPLOAD_DIR, PROJECTS_DIR, EXPORT_DIR]:
    if not os.path.exists(d):
        os.makedirs(d)

# Mounts
app.mount("/uploads", StaticFiles(directory=UPLOAD_DIR), name="uploads")
app.mount("/exports", StaticFiles(directory=EXPORT_DIR), name="exports")
app.mount("/projects", StaticFiles(directory=PROJECTS_DIR), name="projects") # Expose project assets

# Utils
def get_video_duration(file_path):
    try:
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened(): return 0
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        duration = frame_count / fps if fps > 0 else 0
        cap.release()
        return duration
    except Exception as e:
        print(f"Error getting duration for {file_path}: {e}")
        return 0

def get_project_path(project_name):
    # Sanitize project name simple check
    safe_name = "".join(c for c in project_name if c.isalnum() or c in (' ', '_', '-')).strip()
    return os.path.join(PROJECTS_DIR, safe_name)

# --- PROJECTS API ---

class ProjectCreate(BaseModel):
    name: str
    description: Optional[str] = None

@app.get("/api/projects")
def list_projects():
    projects = []
    if os.path.exists(PROJECTS_DIR):
        for dirname in os.listdir(PROJECTS_DIR):
            path = os.path.join(PROJECTS_DIR, dirname)
            if os.path.isdir(path):
                # Try to read manifest
                manifest_path = os.path.join(path, "project.json")
                meta = {}
                if os.path.exists(manifest_path):
                    try:
                        with open(manifest_path, 'r') as f: meta = json.load(f)
                    except: pass
                
                projects.append({
                    "name": dirname,
                    "created_at": meta.get("created_at"),
                    "thumbnail": meta.get("thumbnail"), # Could be path to first video thumb
                    "clip_count": len(os.listdir(os.path.join(path, "source_media"))) if os.path.exists(os.path.join(path, "source_media")) else 0
                })
    return projects

@app.post("/api/projects")
def create_project(data: ProjectCreate):
    print(f"Creating project: {data.name}")
    path = get_project_path(data.name)
    if os.path.exists(path):
        raise HTTPException(status_code=400, detail="Project already exists")
    
    os.makedirs(path)
    os.makedirs(os.path.join(path, "source_media"))
    os.makedirs(os.path.join(path, "exports"))
    
    meta = {
        "name": data.name,
        "description": data.description,
        "created_at": datetime.now().isoformat(),
        "status": "created"
    }
    with open(os.path.join(path, "project.json"), "w") as f:
        json.dump(meta, f)
        
    return meta

@app.delete("/api/projects/{project_name}")
def delete_project(project_name: str):
    path = get_project_path(project_name)
    if not os.path.exists(path):
        raise HTTPException(status_code=404, detail="Project not found")
    
    try:
        shutil.rmtree(path)
        return {"status": "deleted", "name": project_name}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/")
def read_root():
    return {"message": "Gravity Video Editor Backend"}

@app.post("/upload-batch/")
async def upload_batch(
    files: List[UploadFile] = File(...),
    project_name: Optional[str] = Form(None)
):
    uploaded_files = []
    
    # Determine target directory
    if project_name:
        # Sanitize!
        safe_name = "".join(c for c in project_name if c.isalnum() or c in (' ', '_', '-')).strip()
        target_dir = os.path.join(get_project_path(safe_name), "source_media")
        if not os.path.exists(target_dir):
            os.makedirs(target_dir) # Auto create if slightly out of sync
        # Also ensure simple mount access via /projects/name/source_media
        web_path_prefix = f"/projects/{project_name}/source_media"
    else:
        target_dir = UPLOAD_DIR
        web_path_prefix = "/uploads"

    print(f"Received {len(files)} files. Project: {project_name or 'None (Legacy)'}")
    
    for file in files:
        try:
            file_path = os.path.join(target_dir, file.filename)
            with open(file_path, "wb") as buffer:
                shutil.copyfileobj(file.file, buffer)
            
            try:
                # Try getting video duration (opencv)
                duration = get_video_duration(file_path)
            except:
                duration = 0
            
            # If it's 0 and looks like audio, we can try moviepy later or just leave it
            # For now 0 is fine, Frontend handles it (defaulting to 5s visual or looping music)
            
            uploaded_files.append({
                "name": file.filename,
                "path": f"{web_path_prefix}/{file.filename}",
                "duration": duration
            })
            print(f"Successfully uploaded: {file.filename} ({duration}s) to {target_dir}")
        except Exception as e:
            print(f"Failed to upload {file.filename}: {str(e)}")
            pass
            
    return {"files": uploaded_files}

@app.get("/uploaded-videos/")
def list_uploaded_videos(project_name: Optional[str] = None):
    # This might need to support project scoping
    videos = []
    
    if project_name:
        target_dir = os.path.join(get_project_path(project_name), "source_media")
        web_path_prefix = f"/projects/{project_name}/source_media"
    else:
        target_dir = UPLOAD_DIR
        web_path_prefix = "/uploads"
        
    if os.path.exists(target_dir):
        for filename in os.listdir(target_dir):
            if filename.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):
                file_path = os.path.join(target_dir, filename)
                duration = get_video_duration(file_path)
                videos.append({
                    "name": filename,
                    "path": f"{web_path_prefix}/{filename}",
                    "duration": duration
                })
    return {"files": videos}

@app.get("/uploaded-audio/")
def list_uploaded_audio(project_name: Optional[str] = None):
    audio_files = []
    
    if project_name:
        target_dir = os.path.join(get_project_path(project_name), "source_media")
        web_path_prefix = f"/projects/{project_name}/source_media"
    else:
        target_dir = UPLOAD_DIR
        web_path_prefix = "/uploads"
        
    if os.path.exists(target_dir):
        for filename in os.listdir(target_dir):
            if filename.lower().endswith(('.mp3', '.wav', '.aac', '.m4a')):
                file_path = os.path.join(target_dir, filename)
                # duration = get_audio_duration(file_path) # Future
                audio_files.append({
                    "name": filename,
                    "path": f"{web_path_prefix}/{filename}",
                    "type": "audio"
                })
    return {"files": audio_files}

class AnalyzeRequest(BaseModel):
    project_name: str
    file_names: List[str]
    description: Optional[str] = None
    api_key: Optional[str] = None


@app.post("/analyze/")
async def analyze_project(request: AnalyzeRequest):
    # Determine paths based on whether project exists in new structure
    project_path = get_project_path(request.project_name)
    
    # Force output to project directory
    if not os.path.exists(project_path):
        os.makedirs(project_path, exist_ok=True)
    output_dir = project_path

    # PERSIST DESCRIPTION
    if request.description:
        manifest_path = os.path.join(project_path, "project.json")
        meta = {}
        if os.path.exists(manifest_path):
            try:
                with open(manifest_path, 'r') as f: meta = json.load(f)
            except: pass
        
        meta["description"] = request.description
        # ensure other fields exist if creating new
        if "name" not in meta: meta["name"] = request.project_name
        if "created_at" not in meta: meta["created_at"] = datetime.now().isoformat()
        
        with open(manifest_path, "w") as f:
            json.dump(meta, f)

    if os.path.exists(os.path.join(project_path, "source_media")):
        source_dir = os.path.join(project_path, "source_media")
    else:
        # Fallback to uploads if not in new structure yet
        source_dir = UPLOAD_DIR

    # Construct full paths
    video_paths = [os.path.join(source_dir, fname) for fname in request.file_names]
    
    # Verify files exist and fix paths if needed
    verified_paths = []
    for path in video_paths:
        if not os.path.exists(path):
             # Fallback check for legacy mixing
             legacy_path = os.path.join(UPLOAD_DIR, os.path.basename(path))
             if source_dir != UPLOAD_DIR and os.path.exists(legacy_path):
                 verified_paths.append(legacy_path)
             else:
                 raise HTTPException(status_code=404, detail=f"File not found: {path}")
        else:
            verified_paths.append(path)
            
    video_paths = verified_paths

    # Run AI Pipeline via Redis Queue
    if not q_analysis:
        raise HTTPException(status_code=503, detail="Analysis Queue Service Unavailable (Redis)")

    try:
        # Enqueue job
        job = q_analysis.enqueue(
            "backend.worker.tasks.perform_analysis_task",
            video_paths, 
            request.project_name, 
            output_dir, 
            request.description, 
            api_key=request.api_key,
            job_timeout='30m'
        )
        
        print(f"DEBUG: Start Analysis Job {job.id}")
        return {"status": "queued", "job_id": job.id, "message": "AI Analysis started"}
    except Exception as e:
        print(f"Failed to enqueue analysis: {e}")
        raise HTTPException(status_code=500, detail=f"Queue Error: {str(e)}")

@app.get("/analysis-status/{job_id}")
async def get_analysis_status(job_id: str):
    if not redis_conn:
        raise HTTPException(status_code=503, detail="Redis Unavailable")
        
    try:
        job = Job.fetch(job_id, connection=redis_conn)
        status = job.get_status()
        
        # Build response from meta + status
        response = job.meta
        response["status"] = status
        
        # Map RQ statuses to API statuses
        if status == "started": response["status"] = "processing"
        if status == "finished": response["status"] = "completed"
        
        return response
    except NoSuchJobError:
        raise HTTPException(status_code=404, detail="Job not found")
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/api/projects/{project_name}/edl")
async def get_project_edl(project_name: str):
    # Check new project struct ONLY
    project_path = get_project_path(project_name)
    file_path = os.path.join(project_path, f"{project_name}.xml")
    
    if os.path.exists(file_path):
        return FileResponse(file_path, media_type='text/xml', filename=f"{project_name}.xml")
        
    return JSONResponse(status_code=404, content={"detail": f"EDL not found at {file_path}", "cwd": os.getcwd()})

@app.get("/api/projects/{project_name}/analysis")
async def get_project_analysis(project_name: str):
    project_path = get_project_path(project_name)
    file_path = os.path.join(project_path, f"{project_name}_analysis.json")
    
    if os.path.exists(file_path):
        return FileResponse(file_path, media_type='application/json')
    return JSONResponse(status_code=404, content={"detail": "Analysis not found yet"})

class RegenerateRequest(BaseModel):
    instruction: Optional[str] = None
    api_key: Optional[str] = None

@app.post("/api/projects/{project_name}/regenerate-xml")
async def regenerate_project_xml(project_name: str, request: Optional[RegenerateRequest] = None):
    project_path = get_project_path(project_name)
    analysis_path = os.path.join(project_path, f"{project_name}_analysis.json")
    output_xml_path = os.path.join(project_path, f"{project_name}.xml")
    
    if not os.path.exists(analysis_path):
         # Try legacy Uploads path
         legacy_path = os.path.join(UPLOAD_DIR, f"{project_name}_analysis.json")
         if os.path.exists(legacy_path):
             analysis_path = legacy_path
         else:
             raise HTTPException(status_code=404, detail="Analysis file not found. Please run analysis first.")

    try:
        with open(analysis_path, 'r') as f:
            project_data = json.load(f)
            
        # Get Description from project.json if available to pass as context
        user_description = None
        
        # 1. Prefer explicit instruction from request
        if request and request.instruction:
            user_description = request.instruction
        else:
            # 2. Fallback to stored project description
            manifest_path = os.path.join(project_path, "project.json")
            if os.path.exists(manifest_path):
                with open(manifest_path, 'r') as f:
                     meta = json.load(f)
                     user_description = meta.get("description")
        
        api_key_to_use = request.api_key if request else None

        # Regenerate
        success = ai_engine.generate_xml_edl(project_data, output_xml_path, project_name, user_description=user_description, api_key=api_key_to_use)
        
        if not success:
            raise HTTPException(status_code=500, detail="AI Generation Failed. Fallback XML was generated, but AI features (shorts, overlays) are missing. Please check server logs (likely missing GEMINI_API_KEY).")

        return {"status": "success", "message": "XML Regenerated", "path": output_xml_path}
    except Exception as e:
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=str(e))

class ExportRequest(BaseModel):
    project: dict 

@app.post("/export-video/")
async def export_video(request: ExportRequest):
    if not q_render:
         return {"error": "Render Service Unavailable (Redis Queue)"}
    
    try:
        # Enqueue job
        job = q_render.enqueue(
            "backend.worker.tasks.perform_export_task",
            request.project, 
            EXPORT_DIR,
            job_timeout='1h'
        )
        
        # DEBUG: Log payload
        try:
            debug_path = os.path.join(os.getcwd(), "debug_payload.log")
            with open(debug_path, "w") as f:
                clips = request.project.get('clips', [])
                f.write(f"Export Job Enqueued: {job.id}\n")
                f.write(f"Total Clips: {len(clips)}\n")
        except: pass

        return {"status": "queued", "job_id": job.id}
    except Exception as e:
        print(f"Failed to enqueue export: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/export-status/{job_id}")
async def get_export_status(job_id: str):
    if not redis_conn:
        raise HTTPException(status_code=503, detail="Redis Unavailable")
        
    try:
        job = Job.fetch(job_id, connection=redis_conn)
        status = job.get_status()
        
        response = job.meta or {}
        response["status"] = status
        
        if status == "started": response["status"] = "processing"
        if status == "finished": response["status"] = "completed"
        if status == "failed": response["status"] = "failed"
        
        return response
    except NoSuchJobError:
        raise HTTPException(status_code=404, detail="Job not found")
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/cancel-export/{job_id}")
async def cancel_export_job(job_id: str):
    if not redis_conn:
        raise HTTPException(status_code=503, detail="Redis Unavailable")
        
    try:
        job = Job.fetch(job_id, connection=redis_conn)
        job.cancel()
        return {"status": "cancelling", "message": "Job cancellation requested"}
    except NoSuchJobError:
        raise HTTPException(status_code=404, detail="Job not found")
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/api/projects/{project_name}/chat-history")
async def get_project_chat_history(project_name: str):
    project_path = get_project_path(project_name)
    history_file = os.path.join(project_path, "chat_history.json")
    
    if os.path.exists(history_file):
        try:
            with open(history_file, 'r') as f:
                data = json.load(f)
                return data
        except Exception:
            return []
    return []

class ChatRequest(BaseModel):
    query: str
    project_name: str = None
    api_key: Optional[str] = None
    current_state: Optional[Dict[str, Any]] = None
    
@app.post("/chat/")
async def chat_with_ai(request: ChatRequest):
    context = None
    project_path = None
    
    # Determine Project Path and Context
    if request.project_name:
         # Standard Project Path
         p_path = get_project_path(request.project_name)
         project_path = p_path
         
         analysis_path = os.path.join(p_path, f"{request.project_name}_analysis.json")
         # Legacy fallback
         if not os.path.exists(analysis_path):
             legacy_path = os.path.join(UPLOAD_DIR, f"{request.project_name}_analysis.json")
             if os.path.exists(legacy_path):
                 analysis_path = legacy_path
             
         if os.path.exists(analysis_path):
             try:
                 with open(analysis_path, 'r') as f:
                     context = json.load(f)
             except Exception as e:
                 print(f"Failed to load analysis for chat context: {e}")
    else:
        # Default global chat
        project_path = os.path.join(PROJECTS_DIR, "_global_chat")

    # Delegate to Chat Engine (handles LangChain history internally)
    # Pass current_state if provided by frontend
    response = chat_engine.chat(
        request.query, 
        context, 
        project_path=project_path, 
        api_key=request.api_key,
        current_state=request.current_state
    )
    
    return {"response": response}


if __name__ == "__main__":
    import uvicorn
    # uvicorn.run(app, host="0.0.0.0", port=8000)



## 5. Evaluation & Analysis

**Metrics:**
1.  **Transcription Accuracy:** Comparison of 'dirty' vs 'sanitized' transcripts.
2.  **Clip Retention Rate:** The percentage of original footage kept vs. discarded (Target: 30-40% for concise edits).
3.  **Render Integrity:** Success rate of ffmpeg exports without sync issues.

**Sample Output Analysis:**
The system successfully generates an XML EDL (Edit Decision List) which includes:
- `keep='false'` flags for bad audio or silence.
- Color correction tags `<correction type='brightness' value='1.3' />` for dark clips.
- Viral overlay definitions positioned at semantic 'punchlines'.


## 6. Ethical Considerations

**Bias in Editing:**
Automated editing based on 'meaningful speech' carries the risk of erasing hesitations or dialect features that are culturally significant. The prompt explicitly instructs to preserve natural flow but remove 'broken' grammar, which requires careful tuning to avoid linguistic discrimination.

**Content Moderation:**
The system currently processes all inputs. Future improvements must include safety filters (using Gemini's safety settings) to prevent the automated enhancement of harmful or violent content.

**Responsible Use:**
the 'Deepfake' risk is mitigated by only editing *existing* footage rather than generating new pixel data (Video-to-Video), ensuring the subject's likeness remains authentic, merely re-sequenced.


## 7. Conclusion & Future Scope

**Conclusion:**
Gravity Edits demonstrates a viable pipeline for autonomous video post-production. By combining deterministic computer vision metrics with the semantic reasoning of LLMs, we achieve a 'best of both worlds' approach to editing.

**Future Scope:**
1.  **Multi-Modal Agents:** allowing the AI to 'watch' the video (Video-Input LLMs) rather than just reading transcripts.
2.  **Audio Ducking:** Intelligent background music volume automation.
3.  **Style Transfer:** Using GenAI to apply artistic styles to specific clips.
